# GIAI ĐOẠN 1 - CHUẨN BỊ DỮ LIỆU

## IMPORT THƯ VIỆN

In [51]:
import requests
from bs4 import BeautifulSoup
import json
import shutil
import os
import re
import unicodedata
from typing import List, Dict, Any
from datetime import datetime

## CRAWL DỮ LIỆU NGÀNH HỌC

In [52]:
class TDTUMajorCrawler:
    BASE_URL = 'https://admission.tdtu.edu.vn/dai-hoc/nganh-hoc'

    def __init__(self, cache_dir='cache', details_subdir='details'):
        # Tất cả output trung gian sẽ vào cache
        self.cache_dir = cache_dir
        self.output_dir = os.path.join(cache_dir, 'output')
        self.phase1_dir = os.path.join(cache_dir, 'phase1')
        self.details_dir = os.path.join(self.phase1_dir, details_subdir)
        
        # Các file output chính vào cache/output
        self.majors_json_path = os.path.join(self.output_dir, 'tdtu_majors.json')
        self.program_types_path = os.path.join(self.output_dir, 'programme_types.json')
        self.major_detail_out = os.path.join(self.output_dir, 'major-detail.jsonl')

        # Tạo tất cả thư mục cần thiết
        os.makedirs(self.details_dir, exist_ok=True)
        os.makedirs(self.output_dir, exist_ok=True)

    def get_text_safe(self, element):
        return element.get_text(strip=True) if element else ''

    def crawl_major_detail(self, url):
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        data = {}

        data['name'] = self.get_text_safe(soup.find('h2', class_='title'))

        description_block = soup.find('div', class_='block-nganh-1-content')
        data['description'] = '\n'.join(self.get_text_safe(p) for p in description_block.find_all('p')) if description_block else ''

        data['reasons'] = [
            self.get_text_safe(block.select_one('div.highlight_content > div.title'))
            for block in soup.select('div.gsc-column')
            if block.select_one('div.highlight_content > div.title')
        ]

        data['programs'] = []
        tabs_container = soup.select_one('div.gsc-tabs, div.nganh-link-admission')
        if tabs_container:
            tab_list = tabs_container.find('ul')
            if tab_list:
                for li in tab_list.find_all('li'):
                    a = li.find('a')
                    if a:
                        data['programs'].append({
                            'tab': self.get_text_safe(a),
                            'major_code': '',
                            'description': '',
                            'content': {}
                        })

        tab_contents = []
        if soup.select_one('div.gsc-tabs'):
            tab_contents = soup.select('div.gsc-tabs div.tab-content > .tab-pane')
        elif soup.select_one('div.nganh-link-admission'):
            nganh_container = soup.select_one('div.nganh-link-admission')
            tab_contents = [
                div for div in nganh_container.find_all('div', recursive=False)
                if not div.find('ul') and div.get('id')
            ]

        for i, content in enumerate(tab_contents):
            if i >= len(data['programs']):
                continue
            program = data['programs'][i]
            tt_block = content.find('div', class_='tab-chuong-trinh-tt')
            if tt_block:
                strongs = [self.get_text_safe(s) for s in tt_block.find_all('strong') if self.get_text_safe(s)]
                full_text = ' '.join(strongs).strip()
                match = re.search(r'^(.*?)\s*[-–—]?\s*Mã ngành[:：]?\s*([A-Z]{0,2}\s*\d{5,})', full_text, re.IGNORECASE)
                if match:
                    program['name'] = match.group(1).strip()
                    program['major_code'] = match.group(2).replace(" ", "").strip()
                else:
                    program['name'] = full_text

            program['description'] = '\n'.join(self.get_text_safe(p) for p in content.find_all('p') if self.get_text_safe(p))
            ct_block = content.find('div', class_='tab-chuong-trinh')
            if ct_block:
                for section in ct_block.find_all('div', class_='tab-ct'):
                    for flex in section.find_all('div', class_='tab-ct-flex'):
                        title = self.get_text_safe(flex.find('div', class_='tab-ct-flex-title'))
                        if title:
                            value = ' '.join(self.get_text_safe(p) for p in flex.find_all('p'))
                            program['content'][title] = value

        data['programs'] = [p for p in data['programs'] if p['tab'] and (p['description'] or p['content'])]
        data['images'] = [img['src'] for img in soup.select('div.gsc-image img') if img.get('src')]
        return data

    def crawl_all(self):
        result = []
        program_set = set()

        res = requests.get(self.BASE_URL)
        soup = BeautifulSoup(res.content, 'html.parser')
        groups = soup.find_all('div', class_='ts-ng-und')

        for group in groups:
            group_name_tag = group.find(class_='ts-ng-und-td')
            if not group_name_tag:
                continue

            group_name = group_name_tag.get_text(strip=True)
            major_list = []

            for major in group.find_all('a'):
                major_name = major.get_text(strip=True)
                major_link = major['href'].replace('../../../../dai-hoc/nganh-hoc', '').strip()
                full_url = self.BASE_URL + major_link

                info = self.crawl_major_detail(full_url)
                filename = major_link.strip('/').replace('/', '_') + '.json'
                detail_path = os.path.join('details', filename)
                full_detail_path = os.path.join(self.details_dir, filename)

                with open(full_detail_path, 'w', encoding='utf-8') as f:
                    json.dump(info, f, ensure_ascii=False, indent=2)

                for p in info.get('programs', []):
                    if p.get('tab'):
                        program_set.add(p['tab'])

                major_list.append({
                    'name': major_name,
                    'link': major_link,
                    'detail': detail_path
                })

            result.append({
                'group_name': group_name,
                'majors': major_list
            })

        with open(self.majors_json_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)

        with open(self.program_types_path, 'w', encoding='utf-8') as f:
            json.dump(sorted(list(program_set)), f, ensure_ascii=False, indent=2)

        print("=> Crawl hoàn tất. Đã lưu majors và chương trình đào tạo vào thư mục cache/output.")

    def export_jsonl(self):
        detail_files = [f for f in os.listdir(self.details_dir) if f.endswith('.json')]
        detail_files.sort()

        with open(self.major_detail_out, 'w', encoding='utf-8') as out_f:
            for fname in detail_files:
                full_path = os.path.join(self.details_dir, fname)
                with open(full_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    out_f.write(json.dumps(data, ensure_ascii=False) + '\n')
        print(f"=> Đã export {len(detail_files)} ngành vào {self.major_detail_out}")

## CRAWL PHƯƠNG THỨC TUYỂN SINH (DOCUMENT)

In [53]:
class PTTSCrawler:
    def __init__(self, urls, cache_dir="cache"):
        self.urls = urls
        self.cache_dir = cache_dir
        self.json_dir = os.path.join(cache_dir, "ptts")
        self.output_dir = os.path.join(cache_dir, "output")
        self.jsonl_path = os.path.join(self.output_dir, "phuong-thuc.jsonl")
        
        os.makedirs(self.json_dir, exist_ok=True)
        os.makedirs(self.output_dir, exist_ok=True)

    def _fetch_html(self, url):
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        return resp.text

    def _extract_text_content(self, html):
        soup = BeautifulSoup(html, "html.parser")
        content_div = soup.find("div", id="page-main-content")
        if not content_div:
            return None

        for a in content_div.find_all('a', href=True):
            text = a.get_text(strip=True)
            href = a['href']
            replacement = f"({text})[{href}]"
            a.replace_with(replacement)

        return content_div.get_text(separator="\n", strip=True)

    def _save_json(self, name, data):
        json_path = os.path.join(self.json_dir, f"{name}.json")
        with open(json_path, "w", encoding="utf-8") as jf:
            json.dump(data, jf, ensure_ascii=False, indent=2)
        print(f"=> Đã lưu: {json_path}")

    def _append_jsonl(self, data):
        with open(self.jsonl_path, "a", encoding="utf-8") as jsonl_file:
            jsonl_file.write(json.dumps(data, ensure_ascii=False) + "\n")

    def extract_all(self):
        for obj in self.urls:
            url = obj.get("url")
            name = obj.get("name", "output")
            try:
                print(f"=> Đang truy cập: {url}")
                html = self._fetch_html(url)
                text = self._extract_text_content(html)
                if text:
                    data = {
                        "url": url,
                        "name": name,
                        "content": text
                    }
                    self._save_json(name, data)
                    self._append_jsonl(data)
                else:
                    print(f"=> Không tìm thấy nội dung từ {url}")
            except Exception as e:
                print(f"=> Lỗi với {url}: {e}")

## HỌC PHÍ VÀ HỌC BỔNG

In [54]:
HOC_BONG_2023 = """
TRƯỜNG ĐẠI HỌC TÔN ĐỨC THẲNG
CHÍNH SÁCH HỌC BỔNG 2023
TDTU dành hơn 30 tỷ đồng cấp học bổng cho tân sinh viên khóa tuyền sinh 2023
100% học phí
năm học
2023 - 2024
Học bổng thủ khoa
Học bổng dành cho học sinh
các tỉnh hợp tác toàn diện
Học bồng dành cho
học sinh các trường ký kết
Học bổng chương trình
Đại học bằng tiếng Anh
Học bổng ngành thu hút
Học bổng ưu tiên xét tuyền
Học bổng chương trình
Liên kết đào tạo quốc tế
Học bổng
Phân hiệu Khánh Hòa
dành cho tân sinh viên có điểm xét tuyển cao nhất
dành cho học sinh thỏa điều kiện xét học bổng TDTU
của các tỉnh hợp tác toàn diện với Trường
(Bình Thuận, Gia Lai, Bình Định, Lâm Đồng, Quảng Ngãi)
dành cho học sinh các trường THPT có ký kết hợp tác
với TDTU thỏa điều kiện xét học bổng
100% tân sinh viện chương trình dự bị tiếng Anh,
chương trình đại học bằng tiếng Anh được cấp học bổng
theo điều kiện của TDTU
100% tân sinh viên các ngành thu hút
⚫ Công tác xã hội
Công nghệ Kỹ thuật môi trường
Khoa học môi trường
Bảo hộ lao động
Quy hoạch vùng và đô thị
Kỹ thuật xây dựng công trình giao thông
Quản lý thể dục thể thao - Chuyên ngành Golf
được cấp học bổng theo điều kiện của TDTU
100% tân sinh viên nhập học vào các ngành thu hút
theo phương thức ưu tiên xét tuyển của TDTU dành cho
các trường ký kết, có thư giới thiệu của Hiệu trưởng
trường THPT được xét cấp học bổng
100% tân sinh viên chương trình dự bị tiếng Anh,
chương trình Liên kết đào tạo quốc tế được xét cấp
học bồng theo điều kiện của TDTU
100% tân sinh viên học tại Phân hiệu Khánh Hòa
được xét cấp học bổng theo điều kiện của TDTU
Học bổng dành cho học sinh trường chuyên, trường trọng điểm
Học sinh giỏi 03 năm các trường THPT chuyên, trường trọng điểm thỏa điều kiện xét học bổng
Học bổng thế hệ đầu tiên học đại học
Tân sinh viên là thế hệ đầu tiên trong gia đình học đại học trúng tuyền vào các ngành thu hút được
xét cấp học bổng theo điều kiện của TDTU
Các loại học bổng khác
Học bổng cho anh/chị em ruột cùng học tại TDTU
Học bổng cho con, anh/chị/em ruột của cán bộ công đoàn
Học bổng cho sinh viên khuyết tật có hoàn cảnh khó khăn
Học bổng cho tân sinh viên có tiếng Anh đầu vào cao
và nhiều loại học bổng khác
"""

In [55]:
class TuitionScholarshipCrawler:
    def __init__(self, urls, cache_dir="cache"):
        self.urls = urls
        self.cache_dir = cache_dir
        self.output_dir = os.path.join(cache_dir, "output")
        self.output_path = os.path.join(self.output_dir, "hoc-phi-hoc-bong.jsonl")
        
        os.makedirs(self.output_dir, exist_ok=True)

    def _fetch_html(self, url):
        response = requests.get(url, timeout=20)
        response.raise_for_status()
        return BeautifulSoup(response.text, "html.parser")

    def _extract_tables_html(self, parent):
        return [str(table) for table in parent.find_all("table")]

    def _extract_section_content_as_html(self, dd):
        html_parts = []
        for node in dd.children:
            if getattr(node, "name", None) == "table":
                html_parts.append(str(node))
            elif isinstance(node, str):
                text = node.strip()
                if text:
                    html_parts.append(f"<p>{text}</p>")
            elif hasattr(node, "get_text") and node.name != "table":
                clone = node.__copy__()
                for table in clone.find_all("table"):
                    table.extract()
                html_parts.append(str(clone))
        return "\n".join(html_parts).strip()

    def _extract_hoc_phi(self, dl_tag):
        sections = []
        dts = dl_tag.find_all("dt")
        for dt in dts:
            section_title = dt.get_text(strip=True)
            dd = dt.find_next_sibling("dd")
            if dd:
                content = self._extract_section_content_as_html(dd)
                sections.append({
                    "title": section_title,
                    "content": content
                })
        tables_outside = [str(table) for table in dl_tag.find_all("table") if not table.find_parent("dd")]
        return sections, tables_outside

    def _extract_hoc_bong(self, div):
        text = div.get_text(separator="\n", strip=True)
        tables_html = self._extract_tables_html(div)
        content = text
        if tables_html:
            content += "\n" + "\n".join(tables_html)
        return content, tables_html

    def extract_all(self):
        with open(self.output_path, "w", encoding="utf-8") as out_f:
            for obj in self.urls:
                url = obj.get("url")
                name = obj.get("name", "output")
                try:
                    print(f"=> Đang truy cập: {url}")
                    soup = self._fetch_html(url)
                    content_div = soup.find("div", id="page-main-content")
                    if not content_div:
                        print(f"=> Không tìm thấy nội dung ở {url}")
                        continue

                    # Học phí
                    hoc_phi_dl = content_div.find("dl")
                    hoc_phi_sections, hoc_phi_tables = self._extract_hoc_phi(hoc_phi_dl) if hoc_phi_dl else ([], [])

                    # Học bổng
                    hoc_bong_div = content_div.find("div", class_="cshb")
                    hoc_bong_content, hoc_bong_tables = self._extract_hoc_bong(hoc_bong_div) if hoc_bong_div else ("", [])

                    # Ngoại lệ cho 2023
                    if "2023" in name and not hoc_bong_content.strip():
                        hoc_bong_content = HOC_BONG_2023.strip()
                        hoc_bong_tables = []

                    data = {
                        "url": url,
                        "name": name,
                        "hoc_phi": {
                            "sections": hoc_phi_sections,
                            "tables": hoc_phi_tables
                        },
                        "hoc_bong": {
                            "content": hoc_bong_content,
                            "tables": hoc_bong_tables
                        }
                    }
                    out_f.write(json.dumps(data, ensure_ascii=False) + "\n")
                    print(f"=> Đã ghi: {self.output_path}")
                except Exception as e:
                    print(f"=> Lỗi với {url}: {e}")

# GIAI ĐOẠN 2 - CHUẨN HÓA DỮ LIỆU

In [56]:
class TDTUDataConverter:
    def __init__(self, cache_dir: str = "./cache", out_dir: str = "./out"):
        self.data_dir = os.path.join(cache_dir, "output")  # Đọc từ cache/output
        self.out_dir = out_dir
        os.makedirs(self.out_dir, exist_ok=True)

    # ==== Tiện ích chung ====
    @staticmethod
    def remove_accents(text: str) -> str:
        text = unicodedata.normalize('NFD', text)
        return ''.join([c for c in text if unicodedata.category(c) != 'Mn'])

    @classmethod
    def slugify(cls, text: str) -> str:
        text = cls.remove_accents(text)
        text = text.strip().lower()
        text = re.sub(r"\W+", "_", text)
        text = re.sub(r"_+", "_", text)
        return text.strip("_").upper()

    @staticmethod
    def load_jsonl(filename: str) -> List[Dict[str, Any]]:
        with open(filename, 'r', encoding='utf-8') as f:
            return [json.loads(line) for line in f if line.strip()]

    @staticmethod
    def load_json(filename: str) -> Any:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)

    @staticmethod
    def save_json(filename: str, data: Any) -> None:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    # ==== Chuyển đổi chính ====
    def convert_years(self, years: List[int]):
        data = [{"id": str(y), "name": str(y)} for y in years]
        self.save_json(os.path.join(self.out_dir, "years.json"), data)

    def convert_programmes(self, filename: str):
        progs = self.load_json(filename)
        result = [{"id": self.slugify(p.split(":")[0]), "name": p} for p in progs]
        self.save_json(os.path.join(self.out_dir, "programmes.json"), result)

    def convert_majors(self, filename: str):
        major_list = self.load_jsonl(filename)
        majors = [{
            "id": self.slugify(m["name"]),
            "name": m["name"],
            "description": m.get("description", "")
        } for m in major_list]
        self.save_json(os.path.join(self.out_dir, "majors.json"), majors)

    def convert_tuitions(self, hoc_phi_jsonl: str, programme_json: str, years: List[int]):
        prog_map = self._build_programme_title_to_id_map(programme_json)
        data = []
        for doc in self.load_jsonl(hoc_phi_jsonl):
            year_match = re.search(r"\d{4}", doc.get("name", ""))
            year = year_match.group(0) if year_match else ""
            hoc_phi = doc.get("hoc_phi", {})
            for section in hoc_phi.get("sections", []):
                prog_id = self._guess_programme_id(section.get("title", ""), prog_map)
                data.append({
                    "id": f"TUITION_{prog_id}_{year}" if prog_id else f"TUITION_UNKNOWN_{year}",
                    "year_id": year,
                    "programme_id": prog_id,
                    "name": section.get("title", ""),
                    "url": doc.get("url", ""),
                    "content": section.get("content", "")
                })
            if hoc_phi.get("tables"):
                data.append({
                    "id": f"TUITION_OVERVIEW_{year}",
                    "year_id": year,
                    "programme_id": "",
                    "name": f"Tổng quan học phí {year}",
                    "url": doc.get("url", ""),
                    "content": "\n".join(hoc_phi.get("tables", []))
                })
        self.save_json(os.path.join(self.out_dir, "tuitions.json"), data)

    def convert_scholarships(self, hoc_phi_jsonl: str):
        data = []
        for doc in self.load_jsonl(hoc_phi_jsonl):
            year_match = re.search(r"\d{4}", doc.get("name", ""))
            year = year_match.group(0) if year_match else ""
            hoc_bong = doc.get("hoc_bong", {})
            data.append({
                "id": f"SCHOLARSHIPS_{year}",
                "year_id": year,
                "name": doc.get("name", "") + " - Học bổng",
                "url": doc.get("url", ""),
                "content": hoc_bong.get("content", "")
            })
        self.save_json(os.path.join(self.out_dir, "scholarships.json"), data)

    def convert_documents_phuongthuc(self, phuong_thuc_jsonl: str):
        data = []
        for doc in self.load_jsonl(phuong_thuc_jsonl):
            year_match = re.search(r"\d{4}", doc.get("name", ""))
            year = year_match.group(0) if year_match else ""
            data.append({
                "id": f"PHUONG_THUC_{year}",
                "year_id": year,
                "type": "phuong-thuc-tuyen-sinh",
                "name": doc.get("name", ""),
                "url": doc.get("url", ""),
                "html": doc.get("content", ""),
                "text": doc.get("content", "")
            })
        self.save_json(os.path.join(self.out_dir, "documents.json"), data)

    def convert_major_programmes(self, tdtu_majors_json, major_detail_jsonl, programme_types_json, years: List[int]):
        majors_data = self.load_json(tdtu_majors_json)
        major_detail = {self.slugify(m["name"]): m for m in self.load_jsonl(major_detail_jsonl)}
        progs = self.load_json(programme_types_json)
        prog_ids = {p: self.slugify(p.split(":")[0]) for p in progs}
        majors = []
        major_programmes = []

        for group in majors_data:
            for m in group.get("majors", []):
                major_id = self.slugify(m["name"])
                detail = major_detail.get(major_id, {})
                description = detail.get("description", "")
                reasons = detail.get("reasons", [])
                majors.append({
                    "id": major_id,
                    "name": m["name"],
                    "description": description,
                    "reasons": "\n".join(reasons),
                    "images": json.dumps(detail.get("images", []), ensure_ascii=False)
                })

                for prog in detail.get("programs", []):
                    tab = prog.get("tab", "")
                    major_code = prog.get("major_code", "")
                    programme_id = prog_ids.get(tab, "")
                    mp_id = f"{major_id}_{programme_id}"
                    node = {
                        "id": mp_id,
                        "major_id": major_id,
                        "programme_id": programme_id,
                        "tab": tab,
                        "major_code": major_code,
                        "description": prog.get("description", ""),
                        "name": prog.get("name", m["name"]),
                        "year_ids": [str(y) for y in years]
                    }
                    if "content" in prog and isinstance(prog["content"], dict):
                        node.update(prog["content"])
                    major_programmes.append(node)

        self.save_json(os.path.join(self.out_dir, "majors.json"), majors)
        self.save_json(os.path.join(self.out_dir, "major_programmes.json"), major_programmes)

    # ==== Nội bộ ====
    def _build_programme_title_to_id_map(self, programme_json: str) -> Dict[str, str]:
        progs = self.load_json(programme_json)
        mapping = {}
        for p in progs:
            if isinstance(p, dict):
                title = p['name'].split(":")[0].strip().lower()
                prog_id = p['id']
            else:
                title = p.split(":")[0].strip().lower()
                prog_id = self.slugify(title)
            mapping[title] = prog_id
        return mapping

    def _guess_programme_id(self, section_title: str, prog_map: Dict[str, str]) -> str:
        norm_title = self.remove_accents(section_title.lower().strip())
        for prog_title, prog_id in prog_map.items():
            if norm_title.startswith(self.remove_accents(prog_title)):
                return prog_id
        for prog_title, prog_id in prog_map.items():
            if self.remove_accents(prog_title) in norm_title:
                return prog_id
        return ""

    # ==== Chạy toàn bộ ====
    def run_all(self):
        self.convert_years([2023, 2024, 2025])
        self.convert_programmes(os.path.join(self.data_dir, "programme_types.json"))
        self.convert_majors(os.path.join(self.data_dir, "major-detail.jsonl"))
        self.convert_tuitions(
            os.path.join(self.data_dir, "hoc-phi-hoc-bong.jsonl"),
            os.path.join(self.data_dir, "programme_types.json"),
            [2023, 2024, 2025]
        )
        self.convert_scholarships(os.path.join(self.data_dir, "hoc-phi-hoc-bong.jsonl"))
        self.convert_documents_phuongthuc(os.path.join(self.data_dir, "phuong-thuc.jsonl"))
        self.convert_major_programmes(
            os.path.join(self.data_dir, "tdtu_majors.json"),
            os.path.join(self.data_dir, "major-detail.jsonl"),
            os.path.join(self.data_dir, "programme_types.json"),
            [2023, 2024, 2025]
        )
        print(f"=> Dữ liệu đã được xuất ra thư mục {self.out_dir}/")

# GIAI ĐOẠN 3 - IMPORT DỮ LIỆU QUA API

In [57]:
class TDTUDataImporter:
    def __init__(self, api_base: str, email: str, password: str, out_dir: str = "./out"):
        self.api_base = api_base.rstrip("/")
        self.email = email
        self.password = password
        self.out_dir = out_dir
        self.token = None

    def _load_json(self, filename: str) -> Any:
        path = os.path.join(self.out_dir, filename)
        with open(path, encoding="utf-8") as f:
            return json.load(f)

    def _post_json(self, endpoint: str, data: Dict[str, Any]) -> None:
        if not self.token:
            raise Exception("Chưa đăng nhập, vui lòng gọi login() trước khi import")

        url = f"{self.api_base}{endpoint}"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            print(f"[SUCCESS] POST {url} ({response.status_code})")
            print(f"[RESPONSE] {response.json()}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] POST {url} failed: {e}")
            print(f"[RESPONSE TEXT] {response.text}")

    def login(self) -> None:
        """Đăng nhập để lấy token"""
        url = f"{self.api_base}/api/auth/login"
        data = {
            "email": self.email,
            "password": self.password
        }
        try:
            response = requests.post(url, json=data)
            response.raise_for_status()
            result = response.json()
            if result.get("Code") == 1 and "token" in result.get("Data", {}):
                self.token = result["Data"]["token"]
                print("[LOGIN] Đăng nhập thành công.")
            else:
                raise Exception(f"[LOGIN ERROR] Đăng nhập thất bại: {result}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"[LOGIN ERROR] Không thể kết nối tới API: {e}")

    def import_majors_programmes_years(self):
        data = {
            "majors": self._load_json("majors.json"),
            "programmes": self._load_json("programmes.json"),
            "years": self._load_json("years.json"),
            "major_programmes": self._load_json("major_programmes.json")
        }
        self._post_json("/api/v2/import/majors-programmes-years", data)

    def import_tuitions(self):
        data = {
            "tuitions": self._load_json("tuitions.json"),
            "programmes": self._load_json("programmes.json"),
            "years": self._load_json("years.json")
        }
        self._post_json("/api/v2/import/tuitions", data)

    def import_scholarships(self):
        data = {
            "scholarships": self._load_json("scholarships.json"),
            "years": self._load_json("years.json")
        }
        self._post_json("/api/v2/import/scholarships", data)

    def import_documents(self):
        data = {
            "documents": self._load_json("documents.json"),
            "years": self._load_json("years.json")
        }
        self._post_json("/api/v2/import/documents", data)

    def import_all(self):
        print("=> Đăng nhập hệ thống...")
        self.login()
        print("=> Bắt đầu import toàn bộ dữ liệu...")
        self.import_majors_programmes_years()
        self.import_tuitions()
        self.import_scholarships()
        self.import_documents()
        print("=> Hoàn tất import dữ liệu!")

# THỰC THI DỮ LIỆU

In [58]:
# ========== GIAI ĐOẠN 1: CRAWL DỮ LIỆU ==========
print("GIAI ĐOẠN 1: CRAWL DỮ LIỆU")

# 1.1 Crawl ngành học
print("Crawl ngành học...")
major_crawler = TDTUMajorCrawler(cache_dir='cache')
major_crawler.crawl_all()
major_crawler.export_jsonl()

# 1.2 Crawl phương thức tuyển sinh
print("Crawl phương thức tuyển sinh...")
ptts_urls = [
    {
        "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2025",
        "name": "phuong-thuc-2025"
    },
    {
        "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2024",
        "name": "phuong-thuc-2024"
    },
    {
        "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2023",
        "name": "phuong-thuc-2023"
    }
]
ptts_crawler = PTTSCrawler(urls=ptts_urls, cache_dir='cache')
ptts_crawler.extract_all()

# 1.3 Crawl học phí và học bổng
print("Crawl học phí và học bổng...")
tuition_urls = [
    {
        "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2025",
        "name": "hoc-phi-hoc-bong-2025"
    },
    {
        "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2024",
        "name": "hoc-phi-hoc-bong-2024"
    },
    {
        "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2023",
        "name": "hoc-phi-hoc-bong-2023"
    }
]
tuition_crawler = TuitionScholarshipCrawler(urls=tuition_urls, cache_dir='cache')
tuition_crawler.extract_all()

print("Hoàn thành crawl dữ liệu")

# ========== GIAI ĐOẠN 2: CHUẨN HÓA DỮ LIỆU ==========
print("\nGIAI ĐOẠN 2: CHUẨN HÓA DỮ LIỆU")

converter = TDTUDataConverter(cache_dir='cache')
converter.run_all()

print("Hoàn thành chuẩn hóa dữ liệu")

# ========== GIAI ĐOẠN 3: IMPORT DỮ LIỆU ==========
print("\nGIAI ĐOẠN 3: IMPORT DỮ LIỆU")

# Cấu hình API - Thay đổi theo hệ thống của bạn
importer = TDTUDataImporter(
    api_base="http://192.168.1.139:5000",
    email="admin@tdtu.vn",
    password="admin"
)
importer.import_all()

print("Hoàn thành import dữ liệu")
print("\nHOÀN THÀNH TOÀN BỘ QUY TRÌNH!")

GIAI ĐOẠN 1: CRAWL DỮ LIỆU
Crawl ngành học...
=> Crawl hoàn tất. Đã lưu majors và chương trình đào tạo vào thư mục cache/output.
=> Đã export 43 ngành vào cache\output\major-detail.jsonl
Crawl phương thức tuyển sinh...
=> Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2025
=> Đã lưu: cache\ptts\phuong-thuc-2025.json
=> Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2024
=> Đã lưu: cache\ptts\phuong-thuc-2024.json
=> Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2023
=> Đã lưu: cache\ptts\phuong-thuc-2023.json
Crawl học phí và học bổng...
=> Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2025
=> Đã ghi: cache\output\hoc-phi-hoc-bong.jsonl
=> Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2024
=> Đã ghi: cache\output\hoc-phi-hoc-bong.jsonl
=> Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2023
=> Đã ghi: cache\output\h